# <center> Examen de Traitement d'Images</center>

<center>Realisé par : <b>KANNANE Nabil</b></center>

### <center> Détection de la fatigue chez les apprenants </center>

Dans une classe d'apprentissage, certains élèves peuvent se présenter fatigués. Étant donné que la classe est équipée de caméras, nous souhaitons mettre en place un système de détection de la fatigue grâce au calcul de "eye aspect ratio" (EAR). L'idée est de suivre les valeurs de l'EAR au fil du temps et de définir un seuil minimum pour détecter la durée des yeux fermés (indicateur de la fatigue).

### l'importation des bibliotheque

In [8]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import os
import pygame
import csv
import datetime

In [9]:
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("./module/shape_predictor_68_face_landmarks.dat")

In [10]:
pygame.init()
alert_sound = pygame.mixer.Sound("./assets/soundAlert.mp3")

In [11]:
video_path = "./assets/video1.mp4"
output_folder = "./output"

### 1 - Extraction des images à partir d'une vidéo.

In [12]:
def extract_images(video_path, output_folder, num_frames, frame_interval_ms):
    # Ouvrir la vidéo
    vidcap = cv2.VideoCapture(video_path)
    # Définir la durée entre les frames (en millisecondes)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, frame_interval_ms)
    success, image = vidcap.read()
    count = 0
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Parcourir la vidéo et extraire les images avec l'intervalle spécifié entre les frames
    while success and count < num_frames:
        cv2.imwrite(os.path.join(output_folder, f"frame{count}.jpg"), image)     # Sauvegarder l'image dans le dossier de sortie
        count += 1
        vidcap.set(cv2.CAP_PROP_POS_MSEC, frame_interval_ms * (count + 1))  # Avancer la vidéo jusqu'à la prochaine frame à extraire
        success, image = vidcap.read()
    print(f"{count} images extraites avec succès.")

In [13]:
extract_images(video_path, output_folder+"/original",10,1000)

10 images extraites avec succès.


### 2 - Conversion des images en niveaux de gris.

In [14]:
def convert_to_grayscale(input_folder, output_folder):
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Parcourir toutes les images dans le dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Charger l'image en couleur
            img_path = os.path.join(input_folder, filename)
            color_image = cv2.imread(img_path)

            # Convertir l'image en niveaux de gris
            gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

            # Définir le chemin de sortie pour l'image convertie en niveaux de gris
            output_path = os.path.join(output_folder, filename)

            # Écrire l'image convertie en niveaux de gris dans le dossier de sortie
            cv2.imwrite(output_path, gray_image)

    print(f"les images sont converties en niveaux de gris et enregistrée dans {output_folder}.")

In [15]:
convert_to_grayscale(output_folder+"/original",output_folder+"/grayscale")

les images sont converties en niveaux de gris et enregistrée dans ./output/grayscale.


### 3 - Suppression de bruits.

In [16]:
def remove_noise(input_folder, output_folder):
    # Assurer que le dossier de sortie existe, sinon le créer
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Parcourir toutes les images dans le dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Charger l'image en niveaux de gris
            img_path = os.path.join(input_folder, filename)
            gray_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Supprimer le bruit de l'image
            denoised_image = cv2.fastNlMeansDenoising(gray_image, None, 10, 7, 21)

            # Définir le chemin de l'image de sortie
            output_path = os.path.join(output_folder, filename)

            # Écrire l'image débruitée dans le dossier de sortie
            cv2.imwrite(output_path, denoised_image)

            print(f"{filename} sans bruit.")

In [17]:
remove_noise(output_folder+"/grayscale",output_folder+"/remove_noise")

frame0.jpg sans bruit.
frame1.jpg sans bruit.
frame2.jpg sans bruit.
frame3.jpg sans bruit.
frame4.jpg sans bruit.
frame5.jpg sans bruit.
frame6.jpg sans bruit.
frame7.jpg sans bruit.
frame8.jpg sans bruit.
frame9.jpg sans bruit.


### 4 - Amélioration de la qualité des images (contraste, luminosité).

In [18]:
def enhance_quality(input_folder, output_folder, alpha=1.0, beta=0):
    # Assurer que le dossier de sortie existe, sinon le créer
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Parcourir toutes les images dans le dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Charger l'image en niveaux de gris
            img_path = os.path.join(input_folder, filename)
            gray_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Améliorer la qualité de l'image (contraste et luminosité)
            enhanced_image = cv2.convertScaleAbs(gray_image, alpha=alpha, beta=beta)

            # Définir le chemin de l'image de sortie
            output_path = os.path.join(output_folder, filename)

            # Écrire l'image améliorée dans le dossier de sortie
            cv2.imwrite(output_path, enhanced_image)

            print(f"{filename} améliorée.")


In [19]:
enhance_quality(output_folder+"/original", output_folder+"/ameliorer", alpha=1.2, beta=10)

frame0.jpg améliorée.
frame1.jpg améliorée.
frame2.jpg améliorée.
frame3.jpg améliorée.
frame4.jpg améliorée.
frame5.jpg améliorée.
frame6.jpg améliorée.
frame7.jpg améliorée.
frame8.jpg améliorée.
frame9.jpg améliorée.


### 5 - Détection des contours (on utilisant  Canny).

In [20]:
def detect_contours(input_folder, output_folder):
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Parcourir toutes les images dans le dossier d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Charger l'image en niveaux de gris
            img_path = os.path.join(input_folder, filename)
            gray_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Détecter les contours avec l'algorithme de Canny
            edges = cv2.Canny(gray_image, 100, 20)  # Ajustez les seuils 

            # Sauvegarder l'image des contours dans le dossier de sortie
            output_path = os.path.join(output_folder, f"contours_{filename}")
            cv2.imwrite(output_path, edges)

            print(f"Contours détectés pour {filename}.")

In [21]:
detect_contours(output_folder+"/grayscale",output_folder+"/contours")

Contours détectés pour frame0.jpg.
Contours détectés pour frame1.jpg.
Contours détectés pour frame2.jpg.
Contours détectés pour frame3.jpg.
Contours détectés pour frame4.jpg.
Contours détectés pour frame5.jpg.
Contours détectés pour frame6.jpg.
Contours détectés pour frame7.jpg.
Contours détectés pour frame8.jpg.
Contours détectés pour frame9.jpg.


### 6 - Détection des yeux.

In [22]:
def detect_eyes(input_folder, output_folder):
    # Charger le classificateur en cascade pour la détection des visages et des yeux
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Parcourir chaque fichier dans le répertoire d'entrée
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            # Chemin d'accès complet de l'image d'entrée
            input_image_path = os.path.join(input_folder, filename)

            # Charger l'image
            image = cv2.imread(input_image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Détection des visages dans l'image
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            # Pour chaque visage détecté, détecter les yeux
            for (x, y, w, h) in faces:
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = image[y:y+h, x:x+w]
                eyes = eye_cascade.detectMultiScale(roi_gray)

                # Dessiner un rectangle autour de chaque œil détecté
                for (ex, ey, ew, eh) in eyes:
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 1)

            # Sauvegarder l'image avec les rectangles autour des yeux détectés dans le répertoire de sortie
            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, image)

In [23]:
detect_eyes(output_folder+"/grayscale" , output_folder+"/detect_eyes")

### 7 - Application de techniques de morphologie.

In [24]:
def morphology_closure(input_folder , output_folder):
    if not os.path.exists(output_folder): # Créer le dossier de sortie s'il n'existe pas
        os.makedirs(output_folder)
    file_list = os.listdir(input_folder) # Liste des fichiers dans le dossier
    for filename in file_list: # Parcourir chaque fichier dans le dossier
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Vérifier si le fichier est une image
            image_path = os.path.join(input_folder, filename)# Chemin complet de l'image
            image = cv2.imread(image_path)  # Charger l'image
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # Convertir l'image en niveaux de gris si nécessaire
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Définir le kernel
            closed_image = cv2.morphologyEx(gray_image, cv2.MORPH_CLOSE, kernel) # Appliquer une opération de morphologie de fermeture
            # Sauvegarder l'image avec le même nom mais avec un suffixe différent
            output_path = os.path.join(output_folder, filename.split('.')[0] + '_closed.jpg')
            cv2.imwrite(output_path, closed_image)

In [25]:
morphology_closure(output_folder+"/contours", output_folder+"/morphology")

### 8 - Récupération des points d'intérêts (p1,...p6).

In [32]:
def eye_aspect_ratio(eye):
    # calcul de la distance euclidienne entre les deux points verticaux
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    C = distance.euclidean(eye[0], eye[3])  # calcul de la distance euclidienne entre les deux points horizontaux
   
    ear = (A + B) / (2.0 * C) # calcul du ratio des yeux
    
    return ear

In [26]:
def detect_face_landmarks(frame):
    # Convertir la frame en niveau de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Détecter les visages dans la frame
    faces = detect(gray)
    face_landmarks_list = []

    # Processus pour chaque visage détecté dans la frame
    for face in faces:
        face_landmarks = []
        face_landmarks_raw = predict(gray, face)
        for i in range(0, 6):  # Points d'intérêt pour les yeux (p1 à p6)
            x = face_landmarks_raw.part(i).x
            y = face_landmarks_raw.part(i).y
            face_landmarks.append((x, y))
        face_landmarks_list.append(face_landmarks)

    return face_landmarks_list

### 9 - À chaque instant t (par exemple, t=10s), calcul de l'EAR et sauvegarde dans un fichier CSV.

In [41]:
def calculate_ears(all_frames):
    ears_data = []
    for frame in all_frames:
        # Récupérer les points d'intérêt du visage
        face_landmarks_list = detect_face_landmarks(frame)
        for face_landmarks in face_landmarks_list:
            # Calcul de l'EAR
            eye = face_landmarks[0:6]  # Points d'intérêt pour les yeux (p1 à p6)
            ear = eye_aspect_ratio(eye)
            ears_data.append(ear)  # Enregistrer les valeurs EAR

    return ears_data

In [44]:
def split_video_frames(video_path, interval_seconds=0.5):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    # Get the frames per second (fps) 
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the frame interval for every interval_seconds
    frame_interval = int(fps * interval_seconds)
    # List to store selected frames
    selected_frames = []
    # Counter for frame number
    frame_count = 0
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        # Break the loop if we have reached the end of the video
        if not ret:
            break
        # If the frame count is a multiple of frame_interval, add it to the selected frames
        if frame_count % frame_interval == 0:
            selected_frames.append(frame)
        # Increment the frame count
        frame_count += 1
    # Release the video capture object
    cap.release()

    print(selected_frames)
    # Return the selected frames and frame rate
    return selected_frames

In [45]:
def save_to_csv(data):
    timestamp = datetime.datetime.now().strftime("%M:%S:%f")
    with open("ear_data.csv", mode='a', newline='\n') as file:
        writer = csv.writer(file)
        for item in data:
            if file.tell() == 0:
                    writer.writerow(['Time', 'EAR'])
            writer.writerow([timestamp , item])

In [46]:
def Video_detection(video_path):
    # Prendre une vidéo et la diviser en frames
    selected_frames= split_video_frames(video_path)
    # Calculer les données EAR
    ears_data = calculate_ears(selected_frames)    
    # # Enregistrer les données dans les fichiers CSV si des données ont été collectées
    save_to_csv(ears_data)

In [47]:
Video_detection(video_path)

[array([[[136, 143, 163],
        [136, 143, 163],
        [136, 143, 163],
        ...,
        [220, 238, 240],
        [220, 238, 240],
        [220, 238, 240]],

       [[136, 143, 163],
        [136, 143, 163],
        [136, 143, 163],
        ...,
        [220, 238, 240],
        [220, 238, 240],
        [220, 238, 240]],

       [[136, 143, 163],
        [136, 143, 163],
        [136, 143, 163],
        ...,
        [220, 238, 240],
        [220, 238, 240],
        [220, 238, 240]],

       ...,

       [[ 71,  75, 109],
        [ 71,  75, 109],
        [ 72,  76, 110],
        ...,
        [233, 235, 242],
        [233, 235, 242],
        [233, 235, 242]],

       [[ 83,  89, 127],
        [ 83,  89, 127],
        [ 84,  90, 128],
        ...,
        [239, 236, 243],
        [238, 235, 242],
        [238, 235, 242]],

       [[ 91,  97, 135],
        [ 91,  97, 135],
        [ 91,  97, 135],
        ...,
        [239, 236, 243],
        [238, 235, 242],
        [238, 235, 242]

### 10 - Détection de la fatigue d'apres le video

In [34]:
thresh = 0.25
frame_check = 20

In [35]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [36]:
#le chemin de la vidéo
cap = cv2.VideoCapture(video_path)  # Ouvrir la vidéo
# Vos paramètres et fonctions restent les mêmes
flag = 0
while True:
    ret, frame = cap.read()  # Lire chaque frame de la vidéo
    if not ret:  # Verification si la vidéo est terminée
        break
    frame = imutils.resize(frame, width=850) # Redimensionner la frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convertir la frame en niveaux de gris
    subjects = detect(gray, 0) # Détecter les visages dans la frame
    
    # Boucle sur chaque visage détecté dans la frame
    for subject in subjects:
        shape = predict(gray, subject) # Prédire les points d'intérêt du visage
        shape = face_utils.shape_to_np(shape)  # Convertir les points d'intérêt du visage en un format utilisable
        
        # Extraire les points d'intérêt des yeux gauche et droit
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        
        # Calcule l'EAR pour chaque œil
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # Calcule la moyenne de l'EAR pour les deux yeux
        ear = (leftEAR + rightEAR) / 2.0
        
        # Dessine les contours des yeux sur la frame
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # verification si l'EAR est inférieur au seuil de somnolence
        if ear < thresh:
            flag += 1
            #Si la somnolence est détectée pendant un certain nombre de frames consécutives,
            # afficher une alerte et jouer un son d'alerte
            if flag >= frame_check:
                cv2.putText(frame, "ALERT! - Detection de la fatigue", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                alert_sound.play()
        else:
            flag = 0
            
    # Affichage de la frame 
    cv2.imshow("Frame", frame)
    # Attendre une touche pour quitter la boucle
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): 
        break
    
# Libération de la ressource vidéo et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()

#  <center>FIN<center/>